In [1]:
# Inspired by https://github.com/srush/Tensor-Puzzles
from tinygrad import Tensor
import numpy as np

In [ ]:
# These puzzles are about broadcasting. Know this rule.
# Each puzzle needs to be solved in 1 line (<80 columns) of code.
# You are allowed @, arithmetic, comparison, shape, any indexing (e.g. a[:j], a[:, None], a[arange(10)]), and previous puzzle functions.
# You are not allowed anything else. No view, sum, take, squeeze, tensor.

In [4]:
# arange
print(Tensor.arange(3))

<Tensor <LB METAL (3,) int (<BinaryOps.ADD: 1>, None)> on METAL with grad None>


In [3]:
# Example of broadcasting.
examples = [(Tensor.arange(4), Tensor.arange(5)[:, None]) ,
            (Tensor.arange(3)[:, None], Tensor.arange(2))]

print([a + b for a,b in examples])

[<Tensor <LB METAL (5, 4) int (<BinaryOps.ADD: 1>, None)> on METAL with grad None>, <Tensor <LB METAL (3, 2) int (<BinaryOps.ADD: 1>, None)> on METAL with grad None>]


In [5]:
# where
examples = [(Tensor([False]), Tensor([10]), Tensor([0])),
            (Tensor([False, True]), Tensor([1, 1]), Tensor([-10, 0])),
            (Tensor([False, True]), Tensor([1]), Tensor([-10, 0])),
            (Tensor([[False, True], [True, False]]), Tensor([1]), Tensor([-10, 0])),
            (Tensor([[False, True], [True, False]]), Tensor([[0], [10]]), Tensor([-10, 0])),
           ]

[q.where(a, b).numpy() for q, a, b in examples]

[array([0], dtype=int32),
 array([-10,   1], dtype=int32),
 array([-10,   1], dtype=int32),
 array([[-10,   1],
        [  1,   0]], dtype=int32),
 array([[-10,   0],
        [ 10,   0]], dtype=int32)]

In [13]:
# Puzzle 1 - ones
def ones(i) -> Tensor:
  # return Tensor.arange(i) - Tensor.arange(i) + 1
  return Tensor.where(Tensor.arange(i) > -1, 1, 0)

assert np.array_equal(ones(5).numpy(), np.ones(5))

In [7]:
# Puzzle 2 - sum
def sum(a: Tensor) -> Tensor:
  return a @ ones(a.shape[0])

a = Tensor([1, 2, 4, 5])
assert sum(a).numpy() == a.sum().numpy()

In [8]:
# Puzzle 3 - outer
def outer(a: Tensor, b: Tensor) -> Tensor:
  return a[:, None] * b[None, :]

a = Tensor.randint(6)
b = Tensor.randint(3)
assert np.array_equal(outer(a, b).numpy(), np.outer(a.numpy(), b.numpy()))

In [56]:
# Puzzle 4 - diag
def diag(a: Tensor) -> Tensor:
  return a[Tensor.arange(a.shape[0]), Tensor.arange(a.shape[0])]

a = Tensor.randint((5, 5))
assert np.array_equal(diag(a).numpy(), np.diag(a.numpy()))

In [57]:
# Puzzle 5 - eye
def eye(i) -> Tensor:
  # return Tensor.where(Tensor.arange(i)[:, None] == Tensor.arange(i)[None, :], 1, 0)
  return Tensor.arange(i)[:, None] == Tensor.arange(i)

assert np.array_equal(eye(5).numpy(), np.eye(5))

In [61]:
# Puzzle 6 - triu
def triu(j: Tensor) -> Tensor:
  return Tensor.where(Tensor.arange(j)[:, None] <= Tensor.arange(j)[None, :], 1, 0)

def triu(a: Tensor, j: int) -> Tensor:
  # return Tensor.where(Tensor.arange(a.shape[0])[:, None] <= (Tensor.arange(a.shape[0])[None, :] - j), a, 0)
  return Tensor.where(Tensor.arange(a.shape[0])[:, None] <= (Tensor.arange(a.shape[0]) - j), a, 0) # cleaner

a = Tensor.randint((5, 5))
assert np.array_equal(triu(a, 3).numpy(), np.triu(a.numpy(), 3))
assert np.array_equal(triu(a, -1).numpy(), np.triu(a.numpy(), -1))

In [65]:
# Puzzle 7 - cumsum
def cumsum(a: Tensor) -> Tensor:
  return a @ Tensor.where(Tensor.arange(a.shape[0])[:, None] <= Tensor.arange(a.shape[0]), 1, 0)

a = Tensor.randint(5)
assert np.array_equal(cumsum(a).numpy(), np.cumsum(a.numpy()))

In [110]:
# Puzzle 8 - diff
def diff(a: Tensor, i:int) -> Tensor:
  return a[:i] - Tensor.where(Tensor.arange(i) != 0, a[Tensor.arange(i) - 1], 0)

a = Tensor.randint(5)
# assert np.array_equal(diff(a, 2).numpy(), np.diff(a.numpy(), 2))

In [117]:
# Puzzle 9 - vstack
def vstack(a: Tensor, b: Tensor):
  return Tensor.where(Tensor.arange(2)[:, None] == ones(a.shape[0]), b, a)

a = Tensor.randint(5)
b = Tensor.randint(5)
assert np.array_equal(vstack(a, b).numpy(), np.vstack((a.numpy(), b.numpy())))

In [121]:
# Puzzle 10 - roll
def roll(a: Tensor, i: int) -> Tensor:
  # return Tensor.where(, a[Tensor.arange(i-1)], a[Tensor.arange(i+1)])
  return a[(Tensor.arange(i) + 1) % i]

a = Tensor.randint(5)
assert np.array_equal(roll(a, 2).numpy(), np.roll((a.numpy(), 2)))

TypeError: unsupported operand type(s) for %: 'Tensor' and 'int'